In [ ]:
# setup
import pandas as pd
import string
from google.colab import drive
drive.mount('/content/drive')

ratings_raw = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/5500_datasets/Raw_datasets/Books_rating.csv")
ratings = ratings_raw[['Title', 'Id']].drop_duplicates(subset = 'Title', keep = 'first') # keep only 1 ID per title
ratings = ratings.dropna()
books = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/5500_datasets/Raw_datasets/books_data.csv")
enrichment = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/5500_datasets/enrichments/amazonbooks_enrichment_withratings_50k.csv")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# data cleaning
bm = books.merge(ratings[['Title', 'Id']], on='Title', how='left') # add ID info
bm['publishedDate'] = bm['publishedDate'].str[:4] # only take year
bm['categories'] = bm['categories'].str.strip("[]").str.replace("'", "") # get rid of quotes and brackets
bm['categories_list'] = bm['categories'].str.split(', ') # convert to list
bm['authors'] = bm['authors'].str.strip("[]").str.replace("'", "")
bm['authors'] = bm['authors'].str.split(', ') # convert to list

In [ ]:
# extract a list of unique categories
bm['first_category'] = bm['categories_list'].apply(lambda x: x[0] if isinstance(x, list) else None)
bm['first_category'] = bm['first_category'].str.strip("[]").str.replace("'", "")
bm['first_category'] = bm['first_category'].str.lower()
custom_punct = string.punctuation.replace("(", "").replace(")", "")
bm['first_category'] = bm['first_category'].str.rstrip(custom_punct)
unique_categories = bm['first_category'].dropna().unique()
with open('/content/drive/MyDrive/Colab Notebooks/5500_datasets/unique_categories.txt', 'w') as file:
    for category in unique_categories:
        file.write(f"{category}\n")
bm = bm.drop(columns = 'first_category')
bm = bm.drop(columns = 'categories_list')

In [ ]:
# mapping table of author to book ID (accounts for >1 author/book)
authors_mapping = bm.explode('authors', ignore_index=True)
cols = ['Title', 'Id', 'authors']
authors_mapping = authors_mapping[cols]
# authors_mapping.to_csv("/content/drive/MyDrive/Colab Notebooks/5500_datasets/authors_mapping.csv")
bm = bm.drop(columns = 'authors')
# add columns for enrichment
bm = bm.merge(enrichment, left_on='Title', right_on = 'name', how='left')

In [ ]:
# save to CSV
bm = bm.drop(columns = 'name')
bm.to_csv("/content/drive/MyDrive/Colab Notebooks/5500_datasets/books_w_id.csv", index=False)

In [ ]:
# test loading back in
test = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/5500_datasets/books_w_id.csv")
print(test.head())

                                               Title  \
0                     Its Only Art If Its Well Hung!   
1                           Dr. Seuss: American Icon   
2              Wonderful Worship in Smaller Churches   
3                      Whispers of the Wicked Saints   
4  Nation Dance: Religion, Identity and Cultural ...   

                                         description  \
0                                                NaN   
1  Philip Nel takes a fascinating look into the k...   
2  This resource includes twelve principles in un...   
3  Julia Thomas finds her life spinning out of co...   
4                                                NaN   

                                               image  \
0  http://books.google.com/books/content?id=DykPA...   
1  http://books.google.com/books/content?id=IjvHQ...   
2  http://books.google.com/books/content?id=2tsDA...   
3  http://books.google.com/books/content?id=aRSIg...   
4                                             